<a href="https://colab.research.google.com/github/Dutta-SD/NLP/blob/sandip-dev/Aggression_Detection/Experiments/Aggression_Detection_Normal_ML_NEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aggression Experiments

## CODE NOT FULLY FUNCTIONAL. LAST CELL IS UNDER CONSTRUCTION

Aggression_Detection/Experiments/

In [66]:
! pip install -U -qq demoji

In [67]:
import pandas as pd
import numpy as np
from sklearn import (
    metrics, 
    ensemble, 
    svm, 
    feature_extraction, 
    naive_bayes, 
    neural_network, 
    linear_model,
    pipeline,
    manifold,
    preprocessing,
    neighbors,
    gaussian_process
)
import xgboost
from sklearn.model_selection import cross_val_score
import re
import nltk
import string
import demoji

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [68]:
TRAIN_URL_TASK_1 = 'https://raw.githubusercontent.com/Dutta-SD/NLP/master/Aggression_Detection/Aug_Data_Aggression/TASK_A_train_aug_english.csv'
TRAIN_URL_TASK_2 = 'https://raw.githubusercontent.com/Dutta-SD/NLP/master/Aggression_Detection/Aug_Data_Aggression/TASK_B_train_aug_english.csv'

VAL_URL = 'https://raw.githubusercontent.com/Dutta-SD/NLP/master/Aggression_Detection/trac2_eng_dev.csv'

# Task Description
* **A** - Aggression detection
* **B** - Misogyny Detection

In [69]:
# task 1 - Aggression
train = pd.read_csv(TRAIN_URL_TASK_1)
val = pd.read_csv(VAL_URL)
train.head()

,Text,Sub-task A
0,Next part,NAG
1,Iii8mllllllm\nMdxfvb8o90lplppi0005,NAG
2,🤣🤣😂😂🤣🤣🤣😂osm vedio ....keep it up...make more v...,NAG
3,What the fuck was this? I respect shwetabh and...,NAG
4,Concerned authorities should bring arundathi R...,NAG


In [70]:
def seed_all():
  np.random.seed(0)
  
seed_all()

In [71]:
def clean_one_text(text):
    # Cleans one text and returns it    
    
    # remove punctuation
    filter_str = string.punctuation.replace("\'", "")

    new_string = text.translate(str.maketrans('', '', filter_str))
    tk = nltk.TweetTokenizer()

    s = set(nltk.corpus.stopwords.words('english'))
    # n't words
    rexp_1 = re.compile(r"n't")
    not_words = set(filter(rexp_1.findall, s))
    not_words.update(('against', 'no', 'nor', 'not'))

    s.difference_update(not_words)

    # stmr = nltk.stem.snowball.SnowballStemmer("english")
    stmr = nltk.stem.porter.PorterStemmer()
    tokens = [token for token in tk.tokenize(new_string) if token.lower() not in s]
    clean_tokens = [stmr.stem(token) for token in tokens]
    text = ' '.join(clean_tokens)
    # text = demoji.replace_with_desc(text, ':')
    return text

In [72]:
def get_clean_dataset(
    df_raw,
    target_mapping,
    train = True,
    task_name='A', 
    pre_processor=None,
    string_cleaner=None,
    seed = 0):
  '''
  ===============================================================
  get_clean_dataset - cleans the dataset, returns text and labels
  ===============================================================

  :df_raw - pandas dataframe for cleaning
  :target_mapping - map for the targets
  :train - flag to see if training data sent or not
  :task_name - the target to predict
  :preprocessor - preprocesses the string
  :string_cleaner - useful for removing punctuation, etc(function)
  '''

  seed_all()
#   Shuffle
  df_raw = df_raw.sample(frac=1).reset_index()

  col_str = f'Sub-task {task_name}'

  if 'ID' in df_raw.columns:
    df_raw = df_raw.drop(['ID'], axis = 1)

  targets = df_raw[col_str].map(target_mapping).values
  text = df_raw['Text'].values.astype('str')

  if string_cleaner is not None:
    v_cleaner = np.vectorize(string_cleaner)
    text = v_cleaner(text)

  if pre_processor:
    if train:
      text = pre_processor.fit_transform(text)
    else:
      text = pre_processor.transform(text)  

  return text, targets

In [73]:
pp_pipeline = pipeline.Pipeline([
               ('fe', feature_extraction.text.TfidfVectorizer()),
               ('norm', preprocessing.Normalizer()),
               ('pow', preprocessing.QuantileTransformer(output_distribution='normal'))
], verbose=True)
task_1_map ={
    'NAG' : 0,
    'CAG' : 1,
    'OAG' : 2
}

In [74]:
X_train, y_train = get_clean_dataset(
    df_raw = train,
    target_mapping = task_1_map,
    train=True,
    task_name = 'A',
    pre_processor = pp_pipeline,
    string_cleaner = clean_one_text
    
)
X_val, y_val = get_clean_dataset(
    df_raw = val,
    target_mapping = task_1_map,
    train=False,
    task_name = 'A',
    pre_processor = pp_pipeline,
    string_cleaner = clean_one_text
)

[Pipeline] ................ (step 1 of 3) Processing fe, total=   0.2s
[Pipeline] .............. (step 2 of 3) Processing norm, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing pow, total=  10.4s


In [75]:
# count of objects
s = np.bincount(train.iloc[ : , 1].map(task_1_map))

weights = {i : s.sum() / s[i] for i in range(len(s))}

model1 = ensemble.RandomForestClassifier(max_depth = 3,
                                         class_weight=weights, 
                                         random_state=0, 
                                        #  criterion='entropy'
                                         )
model2 = svm.LinearSVC(C = 2, 
                       class_weight=weights, 
                       random_state=0,
                       )
model3 = xgboost.XGBClassifier(max_depth = 4, 
                               gamma = 0.1 ,
                               scale_pos_weight=weights, 
                               random_state=0,
                               n_jobs = -1
                               )
model4 = neural_network.MLPClassifier(hidden_layer_sizes = (256, 128, 256,64),
                                      random_state=0, 
                                      verbose=2, 
                                      learning_rate='adaptive', 
                                      max_iter=20,
                                      tol=1e-3
                                      )
model5 = linear_model.LogisticRegression(
    class_weight = weights,
    random_state = 0,
    max_iter=500,
    n_jobs = -1
    )
model6 = svm.LinearSVC(
    random_state=0
    )
model7 = linear_model.SGDClassifier(
    loss = 'log',
    random_state=0,
    class_weight = weights,
    fit_intercept = False
    )

model_list = [
            #   model1, 
            #   model2, 
              model3, 
            #   model4,
              model5,
            #   model6,
            #   model7,
            ]

In [77]:
# Training Loop -- TASK A

def train_multiple():
    print(f"Preprocessing used : {type(pp_pipeline).__name__}", end="\n\n")

    for i, model in enumerate(model_list):
        print("-"*80)
        print(f"MODEL NO : {i}, TRAINING STARTS...")
        print(f"MODEL NAME : {type(model).__name__}")
        try:
            print(f"MODEL RANDOM SEED(IF ANY) : {model.random_state}")
        except AttributeError:
            pass

        # Fitting
        
        preds = model.fit(X_train, y_train).predict(X_val)
       
        true_preds = y_val


        # Validation
        print(metrics.classification_report(true_preds, preds))
        print(metrics.confusion_matrix(true_preds, preds))

        print(f"\nDone with model {i}")

train_multiple()

Preprocessing used : Pipeline

--------------------------------------------------------------------------------
MODEL NO : 0, TRAINING STARTS...
MODEL NAME : XGBClassifier
MODEL RANDOM SEED(IF ANY) : 0
              precision    recall  f1-score   support

           0       0.88      0.85      0.86       836
           1       0.30      0.41      0.35       117
           2       0.49      0.44      0.46       113

    accuracy                           0.76      1066
   macro avg       0.56      0.57      0.56      1066
weighted avg       0.78      0.76      0.77      1066

[[709  88  39]
 [ 55  48  14]
 [ 41  22  50]]

Done with model 0
--------------------------------------------------------------------------------
MODEL NO : 1, TRAINING STARTS...
MODEL NAME : LogisticRegression
MODEL RANDOM SEED(IF ANY) : 0
              precision    recall  f1-score   support

           0       0.86      0.75      0.80       836
           1       0.30      0.43      0.35       117
           2 

# Stacking
XGB Gives better result than Stacking. Dropping it

In [78]:
# Stacking
estimators = [(f'm{i}', model) for i, model in enumerate(model_list)]

# Stacking Classifier
stk_clf = ensemble.StackingClassifier(
    estimators = estimators,
    n_jobs = -1,
    verbose = True,
)

# Train
stk_clf.fit(X_train, y_train);

In [79]:
## Scoring
y_preds = stk_clf.predict(X_val)
y_true = y_val

print(metrics.classification_report(y_true, y_preds))
print(metrics.confusion_matrix(y_true, y_preds))

              precision    recall  f1-score   support

           0       0.86      0.85      0.86       836
           1       0.33      0.37      0.35       117
           2       0.43      0.40      0.41       113

    accuracy                           0.75      1066
   macro avg       0.54      0.54      0.54      1066
weighted avg       0.76      0.75      0.75      1066

[[714  69  53]
 [ 67  43   7]
 [ 51  17  45]]


In [ ]:
# '''
# DO NOT RUN THIS CELL. UNDER CONSTRUCTION
# '''

# task_2_map ={
#     'NGEN' : 0,
#     'GEN' : 1,
# }
# v1 = feature_extraction.text.CountVectorizer()
# v2 = feature_extraction.text.TfidfVectorizer()

# train_clean = get_clean_dataset(train, True,'B','A', task_2_map, vectorizer=v2)
# val_clean = get_clean_dataset(val, False, 'B', 'A', task_2_map, vectorizer=v2)

# # print(train_clean[1][1:10])
# s_2 = np.bincount(train_clean[1])
# # print(s_2)

# weights_2 = {i : s_2.sum() / s_2[i] for i in range(2)}
# w = s_2[0]/s_2[1]

# model1 = ensemble.RandomForestClassifier(class_weight=weights_2, random_state=0, criterion='entropy')
# model2 = svm.LinearSVC(class_weight=weights_2, random_state=0)
# model3 = xgboost.XGBClassifier(scale_pos_weight=w, random_state=0)
# model4 = neural_network.MLPClassifier(random_state=0, verbose=True, learning_rate='adaptive',max_iter=5 )

# model_list = [model1, model2, model3, model4]

# print(f"Vectorizer used : {type(v2).__name__}", end="\n\n")

# for i, model in enumerate(model_list):
#   print(f"model no {i}, training")
#   print(f"model name {type(model).__name__}")
#   preds = model.fit(train_clean[0], train_clean[1]).predict(val_clean[0])
#   true_preds = val_clean[1]
#   print(metrics.classification_report(true_preds, preds))
#   print(f"\nDone with model {i}")